In [47]:
from simple_wta import WTAProblem, random_wta_factory
import ahuja
import spectral_clustering
import numpy as np

big_prob = random_wta_factory(300,225)
probs,weapons,targets = spectral_clustering.reduce_problem(big_prob,15)
assignments = [ahuja.optimize(p) for p in probs]
values = [probs[i].max_objective(assignments[i]) for i in range(len(probs))]
print(np.sum(values))

c:\Python39\lib\site-packages\numpy\lib\function_base.py:959: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


1087.7


In [48]:
import target_trading
# now we are going to do some trading
# weapon clusters will be randomly chosen to make offers to other clusters.

# everything will be random

for _ in range(100):
    i,j = np.random.choice(len(probs),2,replace=False)
    # need better probability densities than uniform
    stuff = [np.average([big_prob.p[w,t]*big_prob.v[t] for w in weapons[i]]) for t in targets[j]]
    pdf = np.array(stuff)/np.sum(stuff)
    k = np.random.choice(len(targets[j]),p=pdf)
    # get pk data
    pk,v = big_prob.p[weapons[i],targets[j][k]], big_prob.v[targets[j][k]]
    # make offer
    offer = target_trading.make_offer(probs[i].copy(),assignments[i].copy(),pk.copy(),v)
    if len(offer) == 0:
        continue
    t = [targets[i][o[0]] for o in offer]
    pk = big_prob.p[weapons[j],:][:,t]
    v = big_prob.v[t]
    # print(offer)
    response = target_trading.evaluate_offer(
        probs[j].copy(),
        assignments[j].copy(),
        offer,
        k,
        pk.copy(),
        v.copy()
    )
    if response is not None:
        print("huzzah")
        print((i,j,(response,k)))
        u0 = [probs[i].max_objective(assignments[i]),probs[j].max_objective(assignments[j])]
        probs[i].p[:,response] = big_prob.p[weapons[i],targets[j][k]]
        probs[i].v[response] = big_prob.v[targets[j][k]]
        probs[j].p[:,k] = big_prob.p[weapons[j],targets[i][response]]
        probs[j].v[k] = big_prob.v[targets[i][response]]
        big_prob.p[weapons[i],targets[i][response]] = probs[i].p[:,response]
        big_prob.v[targets[i][response]] = probs[i].v[response]
        big_prob.p[weapons[j],targets[j][k]] = probs[j].p[:,k]
        big_prob.v[targets[j][k]] = probs[j].v[k]
        assignments[i] = ahuja.optimize_from_initial(probs[i],assignments[i])
        assignments[j] = ahuja.optimize_from_initial(probs[j],assignments[j])
        print("NEW: ")
        u1 = [probs[i].max_objective(assignments[i]),probs[j].max_objective(assignments[j])]
        print(u1[0]-u0[0]+u1[1]-u0[1])




huzzah
(7, 6, (0, 3))
NEW: 
0.18000000000000682
huzzah
(7, 13, (0, 19))
NEW: 
0.09000000000003183
huzzah
(7, 5, (1, 16))
NEW: 
0.8100000000000023
huzzah
(5, 3, (0, 7))
NEW: 
0.18999999999999773
huzzah
(13, 4, (0, 11))
NEW: 
0.029999999999972715
huzzah
(12, 6, (0, 16))
NEW: 
0.1700000000000017
huzzah
(12, 5, (0, 10))
NEW: 
0.15000000000000568
huzzah
(8, 13, (0, 3))
NEW: 
0.2599999999999909


In [49]:
# assignments = [ahuja.optimize(p) for p in probs]
np.sum([probs[i].max_objective(assignments[i]) for i in range(len(probs))])
# ahuja.optimize(probs[0])

1089.5800000000002